# Use trulens packages from stage

In [ ]:
# Imports.
import shutil
import sys

from snowflake.snowpark.context import get_active_session

# Parameters.
TRULENS_PACKAGES = [
    "trulens-connectors-snowflake",
    "trulens-core",
    "trulens-feedback",
    "trulens-otel-semconv",
    "trulens-providers-cortex",
]
STAGE_NAME = "TRULENS_PACKAGES_STAGE"


# Helper functions.
def clear_existing_trulens_modules() -> None:
    for module_name in list(sys.modules.keys()):
        if module_name.startswith("trulens"):
            del sys.modules[module_name]


def use_package(
    trulens_package: str,
    zip_directory: str = "/tmp/zip",
    expanded_directory: str = "/tmp/expanded",
) -> None:
    snowpark_session = get_active_session()
    get_status = snowpark_session.file.get(
        f"@{STAGE_NAME}/{trulens_package}.zip", zip_directory
    )
    if len(get_status) != 1:
        raise ValueError(f"Failed to download `{trulens_package}.zip`")
    shutil.unpack_archive(
        f"{zip_directory}/{trulens_package}.zip", expanded_directory
    )
    sys.path.insert(0, expanded_directory)


# Use trulens packages from stage.
for trulens_package in TRULENS_PACKAGES:
    use_package(trulens_package)

# Run a simple app

In [ ]:
# Enable OTEL flow.

import os

os.environ["TRULENS_OTEL_TRACING"] = "1"

In [ ]:
# Imports.

import logging
import time
from typing import Any, Sequence
import uuid

from trulens.apps.app import TruApp
from trulens.connectors import snowflake as snowflake_connector
from trulens.core.session import TruSession
from trulens.otel.semconv.trace import SpanAttributes

In [ ]:
# Set up.

logger = logging.getLogger(__name__)
snowpark_session = get_active_session()
db_connector = snowflake_connector.SnowflakeConnector(
    snowpark_session=snowpark_session
)
tru_session = TruSession(db_connector)

In [ ]:
# Create app.

APP_NAME = str(uuid.uuid4())
APP_NAME = (
    APP_NAME.upper()
)  # TODO(this_pr): Remove this requirement or give a better error message!


class MyApp:
    def greet(self, name: str) -> str:
        return f"Hi {name}!"


app = MyApp()
tru_recorder = TruApp(
    app,
    app_name=APP_NAME,
    app_version="v1",
    main_method=app.greet,
)

In [ ]:
# Create run.

from trulens.core.run import RunConfig

run_name = str(uuid.uuid4())
run_config = RunConfig(
    run_name=run_name,
    dataset_name="My test dataframe name",
    source_type="DATAFRAME",
    dataset_spec={"input": "custom_input"},
)

In [ ]:
# Record and invoke.

import pandas as pd

run = tru_recorder.add_run(run_config=run_config)
input_df = pd.DataFrame({"custom_input": ["What is multi-headed attention?"]})
run.start(input_df=input_df)

In [ ]:
# Flush exporter and wait for data to be made to stage.

tru_session.force_flush()

In [ ]:
# Check that the data is in the event table.


def _wait_for_num_results(
    q: str,
    params: Sequence[Any],
    expected_num_results: int,
    num_retries: int = 30,
    retry_cooldown_in_seconds: int = 10,
) -> Sequence:
    for _ in range(num_retries):
        results = snowpark_session.sql(q, params=params).collect()
        if len(results) == expected_num_results:
            return results
        logger.info(
            f"Got {len(results)} results, expecting {expected_num_results}"
        )
        time.sleep(retry_cooldown_in_seconds)
    raise ValueError(
        f"Did not get the expected number of results! Expected {expected_num_results} results, but last found: {len(results)}! The results:\n{results}"
    )


_wait_for_num_results(
    f"""
        SELECT
            *
        FROM
            table(snowflake.local.GET_AI_OBSERVABILITY_EVENTS(
                ?,
                ?,
                ?,
                'EXTERNAL AGENT'
            ))
        WHERE
            RECORD_TYPE = 'SPAN'
            AND TIMESTAMP >= TO_TIMESTAMP_LTZ('2025-01-31 20:42:00')
            AND RECORD_ATTRIBUTES['{SpanAttributes.RUN_NAME}'] = '{run_name}'
        ORDER BY TIMESTAMP DESC
        LIMIT 50
    """,
    [
        snowpark_session.get_current_database()[1:-1],
        snowpark_session.get_current_schema()[1:-1],
        APP_NAME,
    ],
    1,  # TODO(otel): get this from the exporter or something?
)

# Finish with an obvious statement

In [ ]:
print("Kojikun is the world's cutest baby!")